# 1.ライブラリ

In [36]:
import torch
import clip
from PIL import Image
import os
import shutil 

# 2.モデルのロード

In [37]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# 3.分類ラベルの設定
任意の入力が必要です。

In [38]:
# 任意の分類カテゴリ（例：動物分類）
labels = ["human", "a display", "piping","Door","stepladder"]
text_tokens  = clip.tokenize(labels).to(device)

# 4.入力フォルダのパス設定

In [39]:
# 対象画像フォルダのパス（必要に応じて変更）
image_folder = "images"
sorted_folder = "sorted"
os.makedirs(sorted_folder, exist_ok=True)

# 拡張子フィルタ（必要なら追加）
valid_extensions = [".jpg", ".jpeg", ".png", ".bmp"]

# 5.処理内容

In [40]:
# フォルダ内のすべての画像ファイルを処理
for filename in os.listdir(image_folder):
    if not any(filename.lower().endswith(ext) for ext in valid_extensions):
        continue  # 非対応ファイルはスキップ

    image_path = os.path.join(image_folder, filename)
    
    try:
        image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)

        with torch.no_grad():
            logits_per_image, _ = model(image, text_tokens)
            probs = logits_per_image.softmax(dim=-1).cpu().numpy()

        print(f"\n📷 Image: {filename}")
        for label, prob in zip(labels, probs[0]):
            print(f"  {label:<25}: {prob:.4f}")

        best_index = probs[0].argmax()
        predicted_label = labels[best_index]
        print(f"👉 Predicted: {labels[best_index]}")

        # 🔄 画像を分類フォルダにコピー
        label_folder = os.path.join(sorted_folder, predicted_label)
        os.makedirs(label_folder, exist_ok=True)

        # コピー（元画像は残す） → shutil.move にすれば移動
        shutil.copy(image_path, os.path.join(label_folder, filename))
    
    except Exception as e:
        print(f"[Error] Failed to process {filename}: {e}")


📷 Image: 1-1_jimushitu_kyugoshitu_00000.jpg
  human                    : 0.0320
  a display                : 0.9679
  pipes                    : 0.0002
👉 Predicted: a display

📷 Image: 1-1_jimushitu_kyugoshitu_00001.jpg
  human                    : 0.0283
  a display                : 0.9713
  pipes                    : 0.0005
👉 Predicted: a display

📷 Image: 1-1_jimushitu_kyugoshitu_00002.jpg
  human                    : 0.0484
  a display                : 0.9506
  pipes                    : 0.0010
👉 Predicted: a display

📷 Image: 1-1_jimushitu_kyugoshitu_00003.jpg
  human                    : 0.0939
  a display                : 0.9047
  pipes                    : 0.0014
👉 Predicted: a display

📷 Image: 1-1_jimushitu_kyugoshitu_00004.jpg
  human                    : 0.0123
  a display                : 0.9873
  pipes                    : 0.0004
👉 Predicted: a display

📷 Image: 1-1_jimushitu_kyugoshitu_00005.jpg
  human                    : 0.0219
  a display                : 0.9779
  p


KeyboardInterrupt

